### Web Scrapping

In [ ]:
# !pip install --upgrade pip
# !pip install beautifulsoup4 --upgrade
# pip install lxml
# pip install html5lib

In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
import glob

#### External HTML File From Browser Element Section 

In [20]:
lang = "bangla"  # indonesian, turkish, russian, english, urdu, hindi, bangla
surah = "al humaza"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-104"  # al faatiha-1, al baqara-2, aal i imraan-3 ...
file = f"{surah.lower()}_{lang.lower()}"

In [21]:
HTMLFile = open(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Data/{folder.lower().capitalize()}/{file.lower()}.html", "r") # ayahs-block

content = HTMLFile.read()

soup = BeautifulSoup(content, 'lxml')

ayah_word = soup.findAll('span', attrs={"class":"single-word"})

word_list = []
for word in ayah_word:
    var1 = word.find('span', attrs={"class":"word-arabic word-arabic-uthmani-hafs"})
    #try:
    #    #arabic = var1.get_text()
    #    arabic = var1.string
    #except:
    #    arabic = ""
    #arabic = var1.get_text()
    arabic = var1.string
    var2 = word.find('span', attrs={"class":"word-transliteration"})
    transliterate = var2.string
    var3 = word.find('span', attrs={"class":"word-translation"})
    translate = var3.string
    word_list.append((arabic, transliterate, translate))

df_word_list = pd.DataFrame(word_list)
df_word_list.rename(columns={0:"arabic", 1:"transliterate", 2:f"translate_{lang.lower()}"}, inplace=True)
df_word_list

,arabic,transliterate,translate_bangla
0,وَيۡلٞ,waylun,দুর্ভোগ
1,لِّكُلِّ,likulli,প্রত্যেকের
2,هُمَزَةٖ,humazatin,সামনে নিন্দাকারীর
3,لُّمَزَةٍ,lumazatin,পিছনে দোষ প্রচারকারীর
4,ٱلَّذِي,alladhi,যে
5,جَمَعَ,jama'a,জমা করেছে
6,مَالٗا,malan,অর্থ
7,وَعَدَّدَهُۥ,wa'addadahu,এবং তা বারবার গণনা করে রেখেছে
8,يَحۡسَبُ,yahsabu,সে ধারণা করে
9,أَنَّ,anna,যে


In [22]:
df_word_list.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_{lang.lower().capitalize()}.xlsx", index=False)

#### Merge Word All Translate

In [23]:
all_trans_file = glob.glob(f"{surah.lower().capitalize()}*")
all_trans_file

['Al humaza_Arabic_Transliterate_Translate_Words_Indonesian.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_Turkish.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_Russian.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_English.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_Urdu.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_Hindi.xlsx',
 'Al humaza_Arabic_Transliterate_Translate_Words_Bangla.xlsx']

In [24]:
df_master = pd.read_excel(all_trans_file[0])
for i in all_trans_file[1:]:
    df_var = pd.read_excel(f"{i}")
    #df_master = pd.merge(df_master, df_var, how="left", on=["arabic","transliterate"])
    df_master = pd.concat([df_master,df_var.iloc[:,2]], axis=1)
    #df_master = df_master.join(df_var, how="left", lsuffix=["arabic","transliterate"],rsuffix=["arabic","transliterate"])
    #df_master.drop_duplicates(inplace=True)
    df_master.reset_index(drop=True, inplace=True)
df_master = df_master[["arabic","transliterate","translate_english","translate_urdu","translate_hindi","translate_indonesian","translate_bangla","translate_turkish","translate_russian"]]
df_master

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,وَيۡلٞ,waylun,Woe,ہلاکت ہے,हलाकत है,kecelakaan,দুর্ভোগ,vay haline,Горе
1,لِّكُلِّ,likulli,to every,واسطے ہر اس شخص کے,वास्ते हर,bagi setiap,প্রত্যেকের,herkesin,всякому
2,هُمَزَةٖ,humazatin,slanderer,جو منہ پر عیب لگاتا ہے,ऐब लगाने वाले,pengumpat,সামনে নিন্দাকারীর,diliyle çekiştiren,хулителю
3,لُّمَزَةٍ,lumazatin,backbiter!,پیٹھ پیچھے برائیاں کرتا ہے,ग़ीबत करने वाले के,pencela,পিছনে দোষ প্রচারকারীর,kaş göz işaretleriyle alay eden,поносителю.
4,ٱلَّذِي,alladhi,The one who,وہ جس نے,वो जिसने,yang,যে,o ki,который
5,جَمَعَ,jama'a,collects,جمع کیا,जमा किया,mengumpulkan,জমা করেছে,yığdı,собрал
6,مَالٗا,malan,wealth,مال,माल,harta,অর্থ,mal,богатство
7,وَعَدَّدَهُۥ,wa'addadahu,and counts it,اور گن گن کر رکھا اس کو,और उसने गिन-गिन कर रखा उसे,dan dia menghitungnya,এবং তা বারবার গণনা করে রেখেছে,ve onu saydı durdu,и пересчитывал его!
8,يَحۡسَبُ,yahsabu,Thinking,وہ سمجھتا ہے,वो समझता है,dia mengira,সে ধারণা করে,sanıyor,"Думает он,"
9,أَنَّ,anna,that,کہ بیشک,कि बेशक,bahwasanya,যে,muhakkak,что


In [25]:
df_master.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx", index=False)

#### Convert Uthmani To Plain Buckwalter

In [2]:
import pandas as pd
import re
import camel_tools.tokenizers.word
from camel_tools.utils.charmap import CharMapper
from lang_trans.arabic import buckwalter

In [3]:
surah = "al baqara"  # al faatiha, al baqara, aal i imraan ...
folder = f"{surah}-2"  # al faatiha-1, al baqara-2, aal i imraan-3 ...

In [4]:
df_surah = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/{folder.lower().capitalize()}/{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_surah

,arabic,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,الٓمٓ,alif-lam-meem,Alif Laam Meem,الف ۔ لام ۔ میم,अलिफ़ लाम मीम,Alif lam mim,আলিফ লা-ম মী-ম,Elif Lâm Mîm,Алиф лам мим.
1,ذَٰلِكَ,dhalika,That,وہ,ये,itu,(এটা) সেই,işte o,Это
2,ٱلۡكِتَٰبُ,al-kitabu,(is) the Book,کتاب ہے,किताब,Kitab,মহাগ্রন্থ (আল্লাহর),Kitap,Писание
3,لَا,la,no,نہیں,नहीं,tidak ada,নেই,yoktur,нет
4,رَيۡبَۛ,rayba,doubt,کوئی شک,कोई शक,keraguan,কোনো সন্দেহ,hiç şüphe,сомнения
...,...,...,...,...,...,...,...,...,...
6111,مَوۡلَىٰنَا,mawlana,our Protector,ہمارا مولا ہے,मौला है हमारा,penolong kami,আমাদের অভিভাবক,bizim sahibimizsin,покровитель наш
6112,فَٱنصُرۡنَا,fa-unsur'na,so help us,پس مدد فرما ہماری,पस मदद फ़रमा हमारी,maka tolonglah kami,আমাদের সাহায্য কর তাই,bize yardım eyle,помоги же нам
6113,عَلَى,ala,against,اوپر,*,atas,বিরুদ্ধে,karşı,против
6114,ٱلۡقَوۡمِ,al-qawmi,the people,کافر قوم کے,उन लोगों पर,kaum,সম্প্রদায়ের,toplumuna,людей


In [5]:
# Arabic Encoding Extended
# -*- coding: utf-8 -*-

# Arabic Transliteration based on Buckwalter
# dictionary source is buckwalter2unicode.py http://www.redhat.com/archives/fedora-extras-commits/2007-June/msg03617.html 

buck2uni = {"'": u"\u0621", # hamza-on-the-line
            "|": u"\u0622", # madda
            ">": u"\u0623", # hamza-on-'alif
            "&": u"\u0624", # hamza-on-waaw
            "<": u"\u0625", # hamza-under-'alif
            "}": u"\u0626", # hamza-on-yaa'
            "A": u"\u0627", # bare 'alif
            "b": u"\u0628", # baa'
            "p": u"\u0629", # taa' marbuuTa
            "t": u"\u062A", # taa'
            "v": u"\u062B", # thaa'
            "j": u"\u062C", # jiim
            "H": u"\u062D", # Haa'
            "x": u"\u062E", # khaa'
            "d": u"\u062F", # daal
            "*": u"\u0630", # dhaal
            "r": u"\u0631", # raa'
            "z": u"\u0632", # zaay
            "s": u"\u0633", # siin
            "$": u"\u0634", # shiin
            "S": u"\u0635", # Saad
            "D": u"\u0636", # Daad
            "T": u"\u0637", # Taa'
            "Z": u"\u0638", # Zaa' (DHaa')
            "E": u"\u0639", # cayn
            "g": u"\u063A", # ghayn
            "_": u"\u0640", # taTwiil
            "f": u"\u0641", # faa'
            "q": u"\u0642", # qaaf
            "k": u"\u0643", # kaaf
            "l": u"\u0644", # laam
            "m": u"\u0645", # miim
            "n": u"\u0646", # nuun
            "h": u"\u0647", # haa'
            "w": u"\u0648", # waaw
            "Y": u"\u0649", # 'alif maqSuura
            "y": u"\u064A", # yaa'
            "F": u"\u064B", # fatHatayn
            "N": u"\u064C", # Dammatayn
            "K": u"\u064D", # kasratayn
            "a": u"\u064E", # fatHa
            "u": u"\u064F", # Damma
            "i": u"\u0650", # kasra
            "~": u"\u0651", # shaddah
            "o": u"\u0652", # sukuun
            "^": u"\u0653", # maddah
            "#": u"\u0654", # hamzaabove            
            "`": u"\u0670", # dagger 'alif
            "{": u"\u0671", # waSla
            "P": u"\u067E", # arabicletterpeh
            "J": u"\u0686", 
            "V": u"\u06A4", # arabicletterveh
            "G": u"\u06AF", # arabiclettergaf
            ":": u"\u06DC", # smallhighseen
            "@": u"\u06DF", # smallhighroundedzero
            "\"": u"\u06E0", # smallhighuprightrectangularzero
            "[": u"\u06E2", # smallhighmeemisolatedform
            ";": u"\u06E3", # smalllowseen
            ",": u"\u06E5", # smallwaw
            ".": u"\u06E6", # smallya
            "!": u"\u06E8", # smallhighnoon
            "-": u"\u06EA", # emptycentrelowstop
            "+": u"\u06EB", # emptycentrehighstop
            "%": u"\u06EC", # roundedhighstopwithfilledcentre
            "]": u"\u06ED", # smalllowmeem           
}

def transString(string, reverse=0):
    '''Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1'''

    for k, v in buck2uni.items():
      if not reverse:
            string = string.replace(v, k)
      else:
            string = string.replace(k, v)

    return string

In [6]:
def convert_uthmani_to_plain(word):
    buckwalter_letter_select = re.compile(r"[AbptvjHxdrzsSDTZEgfqklmnhwYyFNKaui~o'|>&<}\*\$`]+", re.UNICODE)
    buckwalter_word = transString(word)
    buckwalter_plain = re.findall(buckwalter_letter_select, buckwalter_word)
    word = "".join(buckwalter_plain)

    return word

In [7]:
buckwalter = df_surah.arabic.apply(lambda x: convert_uthmani_to_plain(x))
df_surah.insert(1, "buckwalter_word_tanzil_plain", buckwalter)
df_surah

,arabic,buckwalter_word_tanzil_plain,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,الٓمٓ,Alm,alif-lam-meem,Alif Laam Meem,الف ۔ لام ۔ میم,अलिफ़ लाम मीम,Alif lam mim,আলিফ লা-ম মী-ম,Elif Lâm Mîm,Алиф лам мим.
1,ذَٰلِكَ,*a`lika,dhalika,That,وہ,ये,itu,(এটা) সেই,işte o,Это
2,ٱلۡكِتَٰبُ,lkita`bu,al-kitabu,(is) the Book,کتاب ہے,किताब,Kitab,মহাগ্রন্থ (আল্লাহর),Kitap,Писание
3,لَا,laA,la,no,نہیں,नहीं,tidak ada,নেই,yoktur,нет
4,رَيۡبَۛ,rayba,rayba,doubt,کوئی شک,कोई शक,keraguan,কোনো সন্দেহ,hiç şüphe,сомнения
...,...,...,...,...,...,...,...,...,...,...
6111,مَوۡلَىٰنَا,mawlaY`naA,mawlana,our Protector,ہمارا مولا ہے,मौला है हमारा,penolong kami,আমাদের অভিভাবক,bizim sahibimizsin,покровитель наш
6112,فَٱنصُرۡنَا,fanSurnaA,fa-unsur'na,so help us,پس مدد فرما ہماری,पस मदद फ़रमा हमारी,maka tolonglah kami,আমাদের সাহায্য কর তাই,bize yardım eyle,помоги же нам
6113,عَلَى,EalaY,ala,against,اوپر,*,atas,বিরুদ্ধে,karşı,против
6114,ٱلۡقَوۡمِ,lqawmi,al-qawmi,the people,کافر قوم کے,उन लोगों पर,kaum,সম্প্রদায়ের,toplumuna,людей


In [8]:
#df_surah.to_excel(f"{surah.lower().capitalize()}_Arabic_Transliterate_Translate_Buckwalter_Plain_Word_Languages_Concat.xlsx", index=False)

#### Merge Surah All Translate

In [ ]:
def add_info(df,surah_name, surah_num, index):
    for i in range(len(df)):
        df["surah latin"] = surah_name
        df["surah"] = surah_num
        df.loc[i,"index"] = i+index
    return df

In [ ]:
df_faatir = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Faatir-35/Faatir_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_faatir

In [ ]:
df_faatir = add_info(df_faatir, "Fatir", 35, 56320)
df_faatir = df_faatir.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_faatir

In [ ]:
df_yaseen = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Yaseen-36/Yaseen_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_yaseen

In [ ]:
df_yaseen = add_info(df_yaseen, "Ya Seen", 36, 57102)
df_yaseen = df_yaseen.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_yaseen

In [ ]:
df_saaffaat = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/As Saaffaat-37/As saaffaat_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saaffaat

In [ ]:
df_saaffaat = add_info(df_saaffaat, "As-Saffat", 37, 57836)
df_saaffaat = df_saaffaat.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saaffaat

In [ ]:
df_saad = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Saad-38/Saad_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_saad

In [ ]:
df_saad = add_info(df_saad, "Sad", 38, 58705)
df_saad = df_saad.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_saad

In [ ]:
df_zumar = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Web Scrapping/Result/Az Zumar-39/Az zumar_Arabic_Transliterate_Translate_Words_Languages_Concat.xlsx")
df_zumar

In [ ]:
df_zumar = add_info(df_zumar, "Az-Zumar", 39, 59444)
df_zumar = df_zumar.iloc[:,[10,9,0,1,2,3,4,5,6,7,8,11]]
df_zumar

In [ ]:
df_surah_merge = pd.concat([df_faatir,df_yaseen,df_saaffaat,df_saad,df_zumar], axis=0)
df_surah_merge

In [ ]:
df_surah_merge.to_excel("Faatir_Yaseen_Saaffaat_Saad_Zumar_Surah_Translate_Concat.xlsx", index=False)

#### Test

In [9]:
df_master_all = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quaran/Text/Result/8-Compare Color Flags/Arabic_Buckwalter_All_Relocation_Edit_Counts_Audio_Time_Root_Master.xlsx")
df_master_all

,surah,ayah,word rank,juz,page,surah arabic,surah latin,word count ayah,letter count ayah,tanzil_clean,...,level 1,level 2,level 3,level 4,2gram,3gram,start_time,end_time,repeat flag,index
0,1,1,1,1,1,الفاتحة,Al-Fatiha,4,19,بسم,...,1,2,3,4,2,3,50.0,510.0,NaN,0
1,1,1,2,1,1,الفاتحة,Al-Fatiha,4,19,الله,...,1,2,3,4,2,3,520.0,1180.0,NaN,1
2,1,1,3,1,1,الفاتحة,Al-Fatiha,4,19,الرحمن,...,1,2,3,4,2,3,1190.0,2340.0,NaN,2
3,1,1,4,1,1,الفاتحة,Al-Fatiha,4,19,الرحيم,...,1,2,3,4,2,3,2350.0,4480.0,NaN,3
4,1,2,1,1,1,الفاتحة,Al-Fatiha,4,18,الحمد,...,0,0,2,4,2,0,700.0,1410.0,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78244,114,5,4,30,604,الناس,An-Nas,5,20,صدور,...,0,0,0,0,0,0,5080.0,6760.0,NaN,78244
78245,114,5,5,30,604,الناس,An-Nas,5,20,الناس,...,1,1,1,4,0,0,6770.0,10010.0,NaN,78245
78246,114,6,1,30,604,الناس,An-Nas,3,13,من,...,1,1,1,0,0,0,650.0,1160.0,NaN,78246
78247,114,6,2,30,604,الناس,An-Nas,3,13,الجنة,...,0,1,1,4,0,0,1170.0,3740.0,NaN,78247


In [18]:
df_select_surah = df_master_all[df_master_all.surah == 2]
df_select_surah = df_select_surah.iloc[4:,]
df_select_surah

,surah,ayah,word rank,juz,page,surah arabic,surah latin,word count ayah,letter count ayah,tanzil_clean,...,level 1,level 2,level 3,level 4,2gram,3gram,start_time,end_time,repeat flag,index
33,2,1,1,1,2,البقرة,Al-Baqara,1,3,الم,...,0,0,0,0,0,0,1150.0,6910.0,NaN,33
34,2,2,1,1,2,البقرة,Al-Baqara,7,30,ذلك,...,1,1,1,0,0,0,1310.0,2070.0,NaN,34
35,2,2,2,1,2,البقرة,Al-Baqara,7,30,الكتاب,...,1,1,1,4,0,0,2080.0,3290.0,NaN,35
36,2,2,3,1,2,البقرة,Al-Baqara,7,30,لا,...,1,1,3,0,0,3,3300.0,3740.0,NaN,36
37,2,2,4,1,2,البقرة,Al-Baqara,7,30,ريب,...,0,0,3,0,0,3,3750.0,6100.0,NaN,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169,2,286,45,3,49,البقرة,Al-Baqara,49,204,مولانا,...,0,0,0,4,0,0,77270.0,79550.0,NaN,6169
6170,2,286,46,3,49,البقرة,Al-Baqara,49,204,فانصرنا,...,0,0,0,4,0,0,79560.0,82110.0,NaN,6170
6171,2,286,47,3,49,البقرة,Al-Baqara,49,204,على,...,1,1,1,0,0,0,82120.0,82900.0,NaN,6171
6172,2,286,48,3,49,البقرة,Al-Baqara,49,204,القوم,...,0,1,1,4,0,0,82910.0,84420.0,NaN,6172


In [25]:
df_select_surah.buckwalter_word_tanzil_plain.to_list()

['Alm',
 '*a`lika',
 'AlokitaAbu',
 'laA',
 'rayoba',
 'fiyhi',
 'hudFY',
 'lilomut~aqiyna',
 'Al~a*iyna',
 'yu&ominuwna',
 'biAlogayobi',
 'wayuqiymuwna',
 'AlS~alaApa',
 'wamim~aA',
 'razaqonaAhumo',
 'yunofiquwna',
 'waAl~a*iyna',
 'yu&ominuwna',
 'bimaA',
 '>unozila',
 '<ilayoka',
 'wamaA',
 '>unozila',
 'mino',
 'qabolika',
 'wabiAlo|xirapi',
 'humo',
 'yuwqinuwna',
 '>uwla`}ika',
 'EalaY`',
 'hudFY',
 'mino',
 'rab~ihimo',
 'wa>uwla`}ika',
 'humu',
 'AlomufoliHuwna',
 '<in~a',
 'Al~a*iyna',
 'kafaruwA',
 "sawaA'N",
 'Ealayohimo',
 '>a>ano*arotahumo',
 '>amo',
 'lamo',
 'tuno*irohumo',
 'laA',
 'yu&ominuwna',
 'xatama',
 'All~ahu',
 'EalaY`',
 'quluwbihimo',
 'waEalaY`',
 'samoEihimo',
 'waEalaY`',
 '>aboSaArihimo',
 'gi$aAwapN',
 'walahumo',
 'Ea*aAbN',
 'EaZiymN',
 'wamina',
 'Aln~aAsi',
 'mano',
 'yaquwlu',
 '|man~aA',
 'biAll~ahi',
 'wabiAloyawomi',
 'Alo|xiri',
 'wamaA',
 'humo',
 'bimu&ominiyna',
 'yuxaAdiEuwna',
 'All~aha',
 'waAl~a*iyna',
 '|manuwA',
 'wamaA',
 'yaxodaEuwn

In [20]:
a = set(df_select_surah.buckwalter_word_tanzil_plain.to_list())
a

{'>a*okurokumo',
 'tarotaAbuwA',
 'mulokahu',
 '>anofusikumo',
 'tuxaAliTuwhumo',
 '>adonaY`',
 'AloEaZiymi',
 'Alo>uxoraY`',
 'bijaAluwta',
 'fasayakofiykahumu',
 '>alofa',
 '*ur~iy~apN',
 'mariyDFA',
 'fa>axoraja',
 'saEapF',
 'baEodiy',
 'qaliylFA',
 'wayukaf~iru',
 'Al~a*iy',
 'waAlS~alaApi',
 'yanofaEuhumo',
 '>aEomaAlunaA',
 'litaEotaduwA',
 'wata$aAwurK',
 'wa<isomaAEiyla',
 'waAls~aA}iliyna',
 'yat~aquwna',
 'ta$okuruwna',
 'wa>aqaAmuwA',
 'Alv~amaraAti',
 '>atuwbu',
 'EaAmK',
 'walikul~K',
 'HaniyfFA',
 'wa<i*o',
 'At~ubiEuwA',
 'naHonu',
 '>aHosanu',
 'tat~axi*uwA',
 'yarozuqu',
 'waliy~K',
 'tuHib~uwA',
 'lahumo',
 'wa>anotumo',
 "$uhadaA'akumo",
 '>anoEamotu',
 'Ealima',
 'AlomaSiyru',
 'qawoluhu',
 'Al$~ayoTaAnu',
 'basoTapF',
 'waAsotakobara',
 '>anozalotu',
 'yuxorijo',
 'taSuwmuwA',
 '*ahaba',
 'fajaEalonaAhaA',
 "ha`&ulaA'i",
 'Huduwdu',
 'falaA',
 '>aZolama',
 'wa>ano',
 'AloEa*aAbu',
 'AloHuk~aAmi',
 'linabiy~K',
 '>aEojabakumo',
 'wabaSalihaA',
 'wabay~anuwA',
 'Ean

In [12]:
df_surah

,arabic,buckwalter_word_tanzil_plain,transliterate,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,الٓمٓ,Alm,alif-lam-meem,Alif Laam Meem,الف ۔ لام ۔ میم,अलिफ़ लाम मीम,Alif lam mim,আলিফ লা-ম মী-ম,Elif Lâm Mîm,Алиф лам мим.
1,ذَٰلِكَ,*a`lika,dhalika,That,وہ,ये,itu,(এটা) সেই,işte o,Это
2,ٱلۡكِتَٰبُ,lkita`bu,al-kitabu,(is) the Book,کتاب ہے,किताब,Kitab,মহাগ্রন্থ (আল্লাহর),Kitap,Писание
3,لَا,laA,la,no,نہیں,नहीं,tidak ada,নেই,yoktur,нет
4,رَيۡبَۛ,rayba,rayba,doubt,کوئی شک,कोई शक,keraguan,কোনো সন্দেহ,hiç şüphe,сомнения
...,...,...,...,...,...,...,...,...,...,...
6111,مَوۡلَىٰنَا,mawlaY`naA,mawlana,our Protector,ہمارا مولا ہے,मौला है हमारा,penolong kami,আমাদের অভিভাবক,bizim sahibimizsin,покровитель наш
6112,فَٱنصُرۡنَا,fanSurnaA,fa-unsur'na,so help us,پس مدد فرما ہماری,पस मदद फ़रमा हमारी,maka tolonglah kami,আমাদের সাহায্য কর তাই,bize yardım eyle,помоги же нам
6113,عَلَى,EalaY,ala,against,اوپر,*,atas,বিরুদ্ধে,karşı,против
6114,ٱلۡقَوۡمِ,lqawmi,al-qawmi,the people,کافر قوم کے,उन लोगों पर,kaum,সম্প্রদায়ের,toplumuna,людей


In [26]:
df_surah.buckwalter_word_tanzil_plain.to_list()

['Alm',
 '*a`lika',
 'lkita`bu',
 'laA',
 'rayba',
 'fiyhi',
 'hudY',
 'l~ilmut~aqiyna',
 'l~a*iyna',
 'yu&minuwna',
 'bilgaybi',
 'wayuqiymuwna',
 'lS~alaw`pa',
 'wamim~aA',
 'razaqna`hum',
 'yunfiquwna',
 'wal~a*iyna',
 'yu&minuwna',
 'bimaA',
 '>unzila',
 '<ilayka',
 'wamaA',
 '>unzila',
 'min',
 'qablika',
 'wabil>xirapi',
 'hum',
 'yuwqinuwna',
 '>uwola`}ika',
 'EalaY`',
 'hudY',
 'm~in',
 'r~ab~ihim',
 'wa>uwola`}ika',
 'humu',
 'lmufliHuwna',
 '<in~a',
 'l~a*iyna',
 'kafaruwAo',
 "sawaA'N",
 'Ealayhim',
 "'a>an*artahum",
 '>am',
 'lam',
 'tun*irhum',
 'laA',
 'yu&minuwna',
 'xatama',
 'll~ahu',
 'EalaY`',
 'quluwbihim',
 'waEalaY`',
 'samEihim',
 'waEalaY`',
 '>abSa`rihim',
 'gi$a`wap',
 'walahum',
 'Ea*aAbN',
 'EaZiym',
 'wamina',
 'ln~aAsi',
 'man',
 'yaquwlu',
 "'aAman~aA",
 'bill~ahi',
 'wabilyawmi',
 'l>xiri',
 'wamaA',
 'hum',
 'bimu&miniyna',
 'yuxa`diEuwna',
 'll~aha',
 'wal~a*iyna',
 "'aAmanuwAo",
 'wamaA',
 'yaxdaEuwna',
 '<il~aA',
 '>anfusahum',
 'wamaA',
 'ya$Euruwna

In [21]:
b = set(df_surah.buckwalter_word_tanzil_plain.to_list())
b

{'yaElamuhu',
 'yubay~in',
 'rab~ihim',
 'wa>aqaAmuwAo',
 'EalayhaA',
 'bijaAluwta',
 'qabli',
 'mariyDFA',
 'qaliylFA',
 'yuHib~uwnahum',
 'wayukaf~iru',
 'l~ayli',
 'staEiynuwAo',
 'mavaAbap',
 'yuzak~iyhim',
 'yat~aquwna',
 'walmagfirapi',
 'baEdihi',
 '>atuwbu',
 'maraDA',
 '>aEjabakum',
 'yakfuruwAo',
 'wujuwhakum',
 'faEid~ap',
 'lbaHri',
 'yajEaluwna',
 'wa>aTaEnaA',
 "SafraA'u",
 'yak*ibuwna',
 'Ealima',
 '>aZlama',
 'EaAm',
 'samiEnaA',
 'bill~ahi',
 'walHikmapa',
 'bilba`Tili',
 '<ila`h',
 'bay~an~a`hu',
 '>aSbarahum',
 'lEaliy~u',
 'katabat',
 'taEtaduwhaA',
 'walawlaA',
 'l~itaEtaduwAo',
 '*ahaba',
 "ha`&ulaA'i",
 'hal',
 'naksuwhaA',
 'wabay~ina`t',
 'ta$kuruwna',
 'Huduwdu',
 'falaA',
 'baEDa',
 'l>aswadi',
 "balaA'",
 '>aEma`luhum',
 'EaAd',
 'layaktumuwna',
 'wa<ilayka',
 'tubduwna',
 'xaTa`ya`kum',
 'ls~amiyEu',
 'wabaSalihaA',
 'waDuribat',
 'yaxdaEuwna',
 'lwaSiy~apu',
 'tanZuruwna',
 'yastaHyi',
 'ya>tikum',
 'yuHiyTuwna',
 'mawEiZap',
 'tukal~afu',
 '>uxara',
 'kar

In [24]:
len(b.difference(a))

2025

In [29]:
len(b.intersection(a))

588

In [27]:
df_select_surah.buckwalter_word_tanzil_plain.to_list()

['Alm',
 '*a`lika',
 'AlokitaAbu',
 'laA',
 'rayoba',
 'fiyhi',
 'hudFY',
 'lilomut~aqiyna',
 'Al~a*iyna',
 'yu&ominuwna',
 'biAlogayobi',
 'wayuqiymuwna',
 'AlS~alaApa',
 'wamim~aA',
 'razaqonaAhumo',
 'yunofiquwna',
 'waAl~a*iyna',
 'yu&ominuwna',
 'bimaA',
 '>unozila',
 '<ilayoka',
 'wamaA',
 '>unozila',
 'mino',
 'qabolika',
 'wabiAlo|xirapi',
 'humo',
 'yuwqinuwna',
 '>uwla`}ika',
 'EalaY`',
 'hudFY',
 'mino',
 'rab~ihimo',
 'wa>uwla`}ika',
 'humu',
 'AlomufoliHuwna',
 '<in~a',
 'Al~a*iyna',
 'kafaruwA',
 "sawaA'N",
 'Ealayohimo',
 '>a>ano*arotahumo',
 '>amo',
 'lamo',
 'tuno*irohumo',
 'laA',
 'yu&ominuwna',
 'xatama',
 'All~ahu',
 'EalaY`',
 'quluwbihimo',
 'waEalaY`',
 'samoEihimo',
 'waEalaY`',
 '>aboSaArihimo',
 'gi$aAwapN',
 'walahumo',
 'Ea*aAbN',
 'EaZiymN',
 'wamina',
 'Aln~aAsi',
 'mano',
 'yaquwlu',
 '|man~aA',
 'biAll~ahi',
 'wabiAloyawomi',
 'Alo|xiri',
 'wamaA',
 'humo',
 'bimu&ominiyna',
 'yuxaAdiEuwna',
 'All~aha',
 'waAl~a*iyna',
 '|manuwA',
 'wamaA',
 'yaxodaEuwn

In [28]:
df_surah.buckwalter_word_tanzil_plain.to_list()

['Alm',
 '*a`lika',
 'lkita`bu',
 'laA',
 'rayba',
 'fiyhi',
 'hudY',
 'l~ilmut~aqiyna',
 'l~a*iyna',
 'yu&minuwna',
 'bilgaybi',
 'wayuqiymuwna',
 'lS~alaw`pa',
 'wamim~aA',
 'razaqna`hum',
 'yunfiquwna',
 'wal~a*iyna',
 'yu&minuwna',
 'bimaA',
 '>unzila',
 '<ilayka',
 'wamaA',
 '>unzila',
 'min',
 'qablika',
 'wabil>xirapi',
 'hum',
 'yuwqinuwna',
 '>uwola`}ika',
 'EalaY`',
 'hudY',
 'm~in',
 'r~ab~ihim',
 'wa>uwola`}ika',
 'humu',
 'lmufliHuwna',
 '<in~a',
 'l~a*iyna',
 'kafaruwAo',
 "sawaA'N",
 'Ealayhim',
 "'a>an*artahum",
 '>am',
 'lam',
 'tun*irhum',
 'laA',
 'yu&minuwna',
 'xatama',
 'll~ahu',
 'EalaY`',
 'quluwbihim',
 'waEalaY`',
 'samEihim',
 'waEalaY`',
 '>abSa`rihim',
 'gi$a`wap',
 'walahum',
 'Ea*aAbN',
 'EaZiym',
 'wamina',
 'ln~aAsi',
 'man',
 'yaquwlu',
 "'aAman~aA",
 'bill~ahi',
 'wabilyawmi',
 'l>xiri',
 'wamaA',
 'hum',
 'bimu&miniyna',
 'yuxa`diEuwna',
 'll~aha',
 'wal~a*iyna',
 "'aAmanuwAo",
 'wamaA',
 'yaxdaEuwna',
 '<il~aA',
 '>anfusahum',
 'wamaA',
 'ya$Euruwna